# `latexify`

why pseudocode?

- readability (doesn't break flow of text)
- accessibility (doesn't depend on reader's proficiency with specific language)
- simplicity/abstraction

but we want a reference implementation too

In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
import latexify
import ast

# demo

In [32]:
def collatz(n):
    iterations = 0
    while n > 1:
        if n % 2 == 0:
            n = n // 2
        else:
            n = 3 * n + 1
        iterations = iterations + 1
    return iterations

latexify.algorithmic(collatz)

# plugin system

once complete, implement many of the below as optional plugins

maybe allow users to write custom code at different points in generation _lifecycle_?

- see `get_latex` in [generate_latex.py](../src/latexify/generate_latex.py)
- currently supports custom function calls via `custom_functions` in `visit_Call` in [expression_codegen.py](../src/latexify/codegen/expression_codegen.py)

In [ ]:
%%script true
class MyPlugin(ast.NodeTransformer):
    ...

    def visit_Constant(self, node: ast.Constant):
        ...
        return ... # some latex code

@latexify.algorithmic(plugins=[MyPlugin])
def my_algorithm():
    ...

# or maybe

def customize_my_function(visitor, node):
    return ...  # some latex

@latexify.algorithmic(custom_functions={"my_function": customize_my_function})
def my_algorithm():
    x = my_function()


**Q: best library design patterns for this?**
- what level of flexibility to support?
- what's the point of the library if users just need to write their own AST transformer? (pluggability / modularity)


## docstrings

more description for inputs description, documentation

- i.e. https://peps.python.org/pep-0257/

different docstring styles? maybe require different plugins

enable **comments**. might be via fake docstrings

postprocessing as well as preprocessing (leave "holes" in the tree)

- isn't this just find and replace? How to deal with binding?

substitutions like following:

In [ ]:
%%script true
@latexify.algorithmic(identifiers={"bar": "f"})
def foo():
    x = bar(3)  # not defined

foo._replace(bar=my_replacement)  # but could just walk through the tree again

# type annotations

type checking of shapes and stuff? (maybe tools like `jaxtyping` do this already)

infer which components are vectors or scalars, i.e. shape checking

- support things like `jnp.zeros_like` that need context

function return types

variable type annotations (`AnnAssign` AST)

In [ ]:
x: Float[Array, "N D"] = f(z)

should turn into $\boldsymbol x \in \mathbb R^{N \times D} = \boldsymbol f(\boldsymbol z)$

# other changes

intellisense for frontend (ux), currently just uses `kwargs`. should match kwargs to `get_latex`

In [ ]:
latexify.algorithmic()

dump to `tex` file

In [16]:
@latexify.algorithmic(to_file="square.tex")
def f(x):
    return x**2

turn `elif` into latex `\ElsIf`


In [15]:
def f(x, y):
    if x < y:
        return x
    elif x > y:
        return y

latexify.algorithmic(f)

nested function support

maybe support different `algpseudocode` styles? would need some refactoring

function keyword arguments don't show up

probability notation? e.g. `x = np.random.binomial(n, p)` to $x \sim \text{Binomial}(n, p)$

Typst target? seems worth learning anyways

more advanced identifier replacement
- e.g. `y_hat` to $\widehat y$

In [27]:
def loss(x, y):
    n_batch = len(x)
    y_hat = f(x)
    return (y_hat - y) ** 2 / n_batch

latexify.algorithmic(loss)

In [28]:
latexify.algorithmic(loss, use_math_symbols=True)

support for classes?

other math stuff, e.g. integration, linear algebra?

heterogeneous staging?

variable superscripts as well as subscripts?

In [33]:
def pg_with_learned_baseline_pseudocode(env, pi, eta, theta_init, K, N):
    theta = theta_init
    for k in range(K):
        trajectories = sample_trajectories(env, pi(theta), N)
        V_hat = fit(trajectories)  # estimates the value function of pi(theta)
        tau = sample_trajectories(env, pi(theta), 1)
        nabla_hat = jnp.zeros_like(theta)  # gradient estimator

        for h, (s, a) in enumerate(tau):
            def log_likelihood(theta):
                return jnp.log(pi(theta)(s, a))
            nabla_hat = nabla_hat + jax.grad(log_likelihood)(theta) * (return_to_go(tau, h) - V_hat(s))
        
        theta = theta + eta * nabla_hat
    return theta

latexify.algorithmic(pg_with_learned_baseline_pseudocode, use_math_symbols=True)

$$
\mathrm{log\_likelihood}(\theta) = \log(\pi(\theta)(s, a))
$$